In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn import set_config; set_config(display='diagram')
from tensorflow.keras.preprocessing.text import Tokenizer
import string
import os
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, Sequential
from transformers import AutoTokenizer
from tensorflow.keras.callbacks import EarlyStopping
import datasets

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lucaspicot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#  Load the dataset already cleaned

In [2]:
data = pd.read_csv("/Users/lucaspicot/code/Zen1400/fake_news/raw_data/cleaned_df.csv") #no padding at 500 lenghth

In [3]:
## Delete the first column

data = data.drop(columns= 'Unnamed: 0')

In [4]:
data = data.dropna()

In [5]:
train_val_df = data.sample(frac = 0.8)
test_df = data.drop(train_val_df.index)

train_df = train_val_df.sample(frac = 0.8)
val_df = train_val_df.drop(train_df.index)

In [6]:
df_train = datasets.Dataset.from_pandas(train_df)

In [7]:
df_train

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 67762
})

In [8]:
df_test = datasets.Dataset.from_pandas(test_df)

In [9]:
df_test

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 21176
})

In [10]:
my_dataset_dict = datasets.DatasetDict({"train":df_train,"test":df_test})

# Preprocess

In [11]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

In [13]:
tokenized_data = my_dataset_dict.map(preprocess_function, batched = True)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [15]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 67762
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 21176
    })
})

# Train

In [16]:
tf_train_set = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = tokenized_data["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

2022-09-12 15:30:46.992497: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5

es = EarlyStopping(patience = 5, verbose=2, monitor='val_loss', restore_best_weights = True)

batches_per_epoch = len(tokenized_data["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [28]:
from transformers import TFAutoModelForSequenceClassification


model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
import tensorflow as tf

model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics ='accuracy')

In [30]:
model1.bert.trainable = False

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3,callbacks = [es], batch_size=batch_size)

Epoch 1/3
  11/4236 [..............................] - ETA: 9:36:32 - loss: 1.3769 - accuracy: 0.4943